In [1]:
import re
import os
import sys
import numpy as np
import qiskit
from typing import List
from qiskit import *
# Dynamically add the project root to sys.path
project_root = os.path.abspath(os.path.join(os.path.dirname(os.getcwd())))  # Adjust path to root
sys.path.append(project_root)



In [ ]:
from src.CE_module import construct_qcc_circuit, CE_recur_tree
from src.CA_module import sim_expect_value, update_observables
from src.utilities import compare_lists


In [3]:
#simplified two strings:
test_observables = ['XXXXXX', 'YYYYYY', 'XYXYXY', 'YXYXYX', 'YYYXXX', 'XXXYYY', 'ZZZZZZ', 'ZZIIII', 'IIZZII', 'IIIIZZ', 'XXXXXZ','XXXXZZ', 'XXXZZZ','XXZZZZ','XZZZZZ','ZZZXXX']
test_paulis = ['XXXXXY', 'XXXIYI', 'IXIXXY', 'IXIIYI', 'IXXIXY', 'XXIXYI', 'IIIXIY', 'XIYIII']
test_params = [0.0944527, 0.04799566, -0.0590973, -0.05908328, 0.04114604, 0.02695483, 0.02604318, 0.03485649]

In [4]:
from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers

In [5]:
#first generate the original hamiltonian simulation circuit
origin_qc = construct_qcc_circuit(entanglers = test_paulis, params = test_params, barrier=False)
origin_qc.count_ops()['cx']

40

In [6]:
#generate the qiskit optimized circuit
origin_qiskit = transpile(origin_qc, optimization_level = 3, basis_gates = ["cx", "sx", "x", "rz"])
origin_qiskit.count_ops()['cx']

40

In [7]:
#simulate the circuit and measure the expectation values
orign_expect_vals = []
for obs in test_observables:
    expectation_val =  sim_expect_value(origin_qc, observable=obs, shots=100000)
    orign_expect_vals.append(expectation_val)

XXXXXX
YYYYYY
YXYXYX
XYXYXY
XXXYYY
YYYXXX
ZZZZZZ
ZZZZZZ
ZZZZZZ
ZZZZZZ
ZXXXXX
ZZXXXX
ZZZXXX
ZZZZXX
ZZZZZX
XXXZZZ


In [8]:
print(orign_expect_vals)

[0.09324, -0.09074, 0.0047, -0.00134, -0.0011, -0.00514, 1.0, 0.99476, 0.99484, 0.99646, 0.00114, -0.00026, -0.00024, 0.0002, 0.0049, -0.00456]


In [9]:
#Then we run QuCLEAR to optimize the circuit
opt_qc, append_clifford, sorted_entanglers = CE_recur_tree(entanglers=test_paulis, params=test_params, barrier=False)


In [10]:
#QuCLEAR generates the optimized circuit with reduced number of CNOT gates
opt_qc.count_ops()['cx']

17

In [11]:
append_clifford.count_ops()

OrderedDict([('cx', 20), ('h', 18), ('s', 9), ('x', 4), ('swap', 2), ('y', 1)])

In [12]:
append_clifford.draw()

┌───┐┌───┐                  ┌───┐┌───┐     ┌───┐┌───┐     »
q_0: ────────────■──┤ H ├┤ H ├──────X───────────┤ X ├┤ H ├──■──┤ H ├┤ S ├─────»
               ┌─┴─┐├───┤├───┤┌───┐ │      ┌───┐└─┬─┘└───┘  │  └───┘├───┤┌───┐»
q_1: ───────■──┤ X ├┤ H ├┤ S ├┤ H ├─┼──────┤ X ├──┼─────────┼────■──┤ S ├┤ H ├»
     ┌───┐  │  └───┘└───┘└───┘└───┘ │ ┌───┐└─┬─┘  │       ┌─┴─┐┌─┴─┐└───┘├───┤»
q_2: ┤ H ├──┼───────────────────────X─┤ X ├──┼────■───────┤ X ├┤ X ├──■──┤ X ├»
     └───┘┌─┴─┐┌───┐┌───┐             └─┬─┘  │            └───┘└───┘┌─┴─┐├───┤»
q_3: ──■──┤ X ├┤ S ├┤ H ├───────────────┼────■──────────────────────┤ X ├┤ S ├»
       │  ├───┤└───┘└───┘               │                           └───┘└───┘»
q_4: ──┼──┤ H ├─────────────────────────┼─────────────────────────────────────»
     ┌─┴─┐├───┤┌───┐┌───┐               │                                     »
q_5: ┤ X ├┤ H ├┤ S ├┤ H ├───────────────■─────────────────────────────────────»
     └───┘└───┘└───┘└───┘                                                     »
«     ┌───┐                    ┌───┐                            ┌───┐┌───┐     »
«q_0: ┤ X ├───────■────────────┤ X ├───────■─────────────X──────┤ X ├┤ H ├──■──»
«     └─┬─┘       │  ┌───┐┌───┐└─┬─┘     ┌─┴─┐     ┌───┐ │      └─┬─┘└───┘  │  »
«q_1: ──■─────────┼──┤ X ├┤ H ├──┼────■──┤ X ├──■──┤ X ├─┼────────┼─────────┼──»
«                 │  └─┬─┘└───┘  │    │  └───┘  │  └───┘ │        │         │  »
«q_2: ────────────┼────┼─────────┼────┼─────────┼────────┼────────┼─────────┼──»
«     ┌───┐┌───┐┌─┴─┐  │  ┌───┐  │    │         │        │        │         │  »
«q_3: ┤ H ├┤ X ├┤ X ├──■──┤ Y ├──┼────┼─────────┼────────┼────────┼─────────┼──»
«     └───┘└─┬─┘└───┘     └───┘  │  ┌─┴─┐       │        │ ┌───┐  │  ┌───┐  │  »
«q_4: ───────┼───────────────────┼──┤ X ├───────┼────────┼─┤ X ├──┼──┤ X ├──┼──»
«            │  ┌───┐┌───┐       │  └───┘     ┌─┴─┐┌───┐ │ └─┬─┘  │  └───┘┌─┴─┐»
«q_5: ───────■──┤ S ├┤ H ├───────■────────────┤ X ├┤ H ├─X───■────■───────┤ X ├»
«               └───┘└───┘                    └───┘└───┘                  └───┘»
«     ┌───┐┌───┐┌───┐
«q_0: ┤ S ├┤ H ├┤ S ├
«     └───┘└───┘└───┘
«q_1: ───────────────
«                    
«q_2: ───────────────
«                    
«q_3: ───────────────
«                    
«q_4: ───────────────
«     ┌───┐          
«q_5: ┤ X ├──────────
«     └───┘

In [13]:
# based on the observables and the appended clifford circuit, we can calculate the updated observables
updated_signs, updated_observables = update_observables(test_observables, [append_clifford])
print(updated_observables)

['XIIYZZ', 'XXIYZI', 'XXZXZI', 'XIZXZZ', 'XXZXZZ', 'XIZXZI', 'IXIIIZ', 'ZXXXII', 'XIYXXI', 'YIZIXZ', 'XIIYZY', 'XZYYYY', 'ZZXYIZ', 'IZIYYY', 'XXIXXY', 'YYXIZZ']


In [14]:
len(updated_observables)

16

In [15]:
len(updated_signs)

16

In [16]:
#Run the same circuit simulation process for the updated observables
opt_expect_vals = []
for idx, obs in enumerate(updated_observables):
    expectation_val = sim_expect_value(opt_qc, observable=obs, shots=1000000)
    if updated_signs[idx] == '+1':
        updated_sign = 1
    elif updated_signs[idx] == '-1':
        updated_sign = -1
    else:
        raise Exception("incorrect sign")
    opt_expect_vals.append(updated_sign * expectation_val)

ZZYZZX
ZZYZXX
ZZXZXX
ZZXZZX
ZZXZXX
ZZXZZX
ZZZZXZ
ZZXXXZ
ZXXYZX
ZXZZZY
YZYZZX
YYYYZX
ZZYXZZ
YYYZZZ
YXXZXX
ZZZXYY


In [17]:
opt_expect_vals

[0.093174,
 -0.092354,
 -0.000252,
 3.4e-05,
 0.001118,
 0.0012,
 1.0,
 0.994754,
 0.99471,
 0.996588,
 0.000118,
 -6.2e-05,
 -0.00028,
 0.001616,
 0.000586,
 0.000858]

In [18]:
orign_expect_vals

[0.09324,
 -0.09074,
 0.0047,
 -0.00134,
 -0.0011,
 -0.00514,
 1.0,
 0.99476,
 0.99484,
 0.99646,
 0.00114,
 -0.00026,
 -0.00024,
 0.0002,
 0.0049,
 -0.00456]

Validate the results

In [19]:
try:
    compare_lists(orign_expect_vals, opt_expect_vals, tolerance = 0.01)
    print("Lists differences are within acceptable limits.")
except ValueError as e:
    print(e)

Lists differences are within acceptable limits.
